<a href="https://colab.research.google.com/github/plue1011/GNN/blob/master/RecSys2015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.5
    Uninstalling torch-scatter-2.0.5:
      Successfully uninstalled torch-scatter-2.0.5


Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-sparse 0.6.7
    Uninstalling torch-sparse-0.6.7:
      Successfully uninstalled torch-sparse-0.6.7


Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_cluster-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-cluster 1.5.7
    Uninstalling torch-cluster-1.5.7:
      Successfully uninstalled torch-cluster-1.5.7


Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_spline_conv-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-spline-conv 1.2.0
    Uninstalling torch-spline-conv-1.2.0:
      Successfully uninstalled torch-spline-conv-1.2.0


In [5]:
%cd /content/drive/My\ Drive/fueki/GNN

/content/drive/My Drive/fueki/GNN


In [6]:
import pandas as pd
import numpy as np

import torch
from torch_geometric.data import Data

from sklearn.preprocessing import LabelEncoder

## グラフ変換前のデータ

In [7]:
df = pd.read_csv('./dataset/RecSys/yoochoose-clicks.dat', header=None)
df.columns=['session_id', 'timestamp', 'item_id', 'category']

buy_df = pd.read_csv('./dataset/RecSys/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id', 'timestamp', 'item_id', 'price', 'quantity']

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
np.random.seed(0)
sampled_session_id = np.random.choice(df.session_id.unique(), 100, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)].reset_index(drop=True)
buy_df = buy_df.loc[buy_df.session_id.isin(df.session_id)].reset_index(drop=True)
df.nunique()

session_id    100
timestamp     316
item_id       258
category       14
dtype: int64

In [9]:
item_encoder = LabelEncoder()
category_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id)
df['category']= category_encoder.fit_transform(df.category.apply(str))
buy_df['item_id'] = item_encoder.transform(buy_df.item_id)

In [10]:
df.head()

,session_id,timestamp,item_id,category
0,236826,2014-04-06T10:13:23.153Z,140,0
1,236826,2014-04-06T10:15:18.247Z,19,0
2,236826,2014-04-06T10:16:15.955Z,105,0
3,339162,2014-04-06T18:23:48.779Z,106,0
4,339162,2014-04-06T18:25:03.122Z,63,0


In [11]:
buy_df.head()

,session_id,timestamp,item_id,price,quantity
0,1283789,2014-04-21T03:17:14.846Z,121,732,1
1,1283789,2014-04-21T03:17:14.914Z,124,271,1
2,1283789,2014-04-21T03:17:14.931Z,100,1046,1
3,1283789,2014-04-21T03:17:14.980Z,119,1046,1
4,2045783,2014-05-04T06:49:05.960Z,97,0,0


In [48]:
class RecSysDataset(InMemoryDataset): 
    def __init__(self, root, name, df, buy_df, transform=None, pre_transform=None):
        self.root = root
        self.name = name
        self.df = df
        self.buy_df = buy_df
        super(RecSysDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return [self.root + self.name]

    def download(self):
        pass
    
    def process(self):
        data_list = []

        buy_id = np.unique(self.buy_df['session_id'].values)
        for session, group in self.df.groupby('session_id'):
            group = group.copy()
            node_id_encoder = LabelEncoder()
            nodes = node_id_encoder.fit_transform(group.item_id)
            group['node_id'] = nodes

            # node feature
            node_features = group[['item_id', 'node_id']].sort_values('node_id')[['item_id']].drop_duplicates().values
            x = torch.tensor(node_features, dtype=torch.long)

            # edge
            from_nodes = nodes[:-1]
            to_nodes = nodes[1:]
            edge_index = torch.tensor([from_nodes, to_nodes], dtype=torch.long)

            # target
            target = np.zeros(len(node_features))
            if session in buy_id:
                positive_flag = node_id_encoder.transform(self.buy_df.query('session_id == @session')['item_id'])
                target[positive_flag] = 1
            y = torch.tensor(target, dtype=torch.long)

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [51]:
dataset = RecSysDataset(root='/content/drive/My Drive/fueki/GNN/dataset/RecSys', 
                        name='/test.dataset',
                        df=df, 
                        buy_df=buy_df)
dataset[3].x

tensor([[ 81],
        [175]])

## Datasetの作成方法

In [24]:

d = ZINC(root='./sample_data/')

Extracting sample_data/molecules.zip?dl=1
Processing...
ssssssssssss


Processing test dataset: 100%|██████████| 5000/5000 [00:00<00:00, 16939.11it/s]


Done!


In [25]:
import os
os.path.join('./', 'test', 'processed')

'./test/processed'